In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.datasets import make_regression

n_samples = 1000
n_features = 10
n_informative = 3

X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative)

# We convert the X and y arrays into a DataFrame which will be useful later
col_names = []
for i in range(n_features):
    col_names.append("X{}".format(i+1))

df = pd.DataFrame(X, columns=col_names)
df["Y"] = y
display(df)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,1.261356,-0.097358,1.294245,0.543862,-0.502822,-1.281723,-0.046621,2.415792,0.068971,-0.603129,-69.010727
1,0.730212,-1.304883,-0.568725,0.406990,0.084467,-0.061067,1.323982,0.242936,-0.829721,0.256949,99.441625
2,0.529823,-0.338146,1.257167,1.602550,1.153299,0.673513,-0.087808,-0.026511,-0.913234,0.411988,57.973794
3,0.493558,0.999604,-0.158930,-0.022898,0.255797,0.565085,0.298400,0.877258,-2.134192,-1.908179,-153.483517
4,0.624281,-0.540913,-1.799899,-0.297454,-1.549862,-4.486307,0.009664,-0.644910,1.440359,0.136770,-20.393197
...,...,...,...,...,...,...,...,...,...,...,...
995,1.200977,-0.983135,-0.191009,-0.916609,-1.392736,-2.587765,1.542102,-0.133879,-0.277060,1.079835,155.434904
996,2.150441,0.417415,-0.201864,0.376407,1.661188,1.105260,-0.483471,-0.934109,0.571600,0.308711,37.349797
997,-1.794214,0.308991,-0.290869,-0.122808,-0.670278,-0.466196,0.146303,0.776115,-1.268303,-0.438614,-46.709404
998,-1.025510,0.695811,-0.130971,0.723426,-0.925012,0.352238,0.945332,1.858181,-0.237365,0.868465,112.756103


In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor()
RFR.fit(X, y)

In [ ]:
df_feature_importances = pd.DataFrame(RFR.feature_importances_,
                                      columns=["Importance"],
                                      index=col_names)
df_feature_importances = df_feature_importances.sort_values("Importance", ascending=False)
df_feature_importances

In [ ]:
# We store the describe() results inside a dataframe
df_describe = df.describe()
display(df_describe)

# We define the parameters of the virtual population we generate
population_size = 1000
features_names  = df.columns[:-1]
n_features = len(features_names)

# As an example, we assign a constant value for the third most important characteristic
constraint_feature = df_feature_importances.index[2]
constraint_feature_value = round(np.random.uniform(df_describe.loc["min",constraint_feature], df_describe.loc["max",constraint_feature]),3)
print("\n", constraint_feature_value, "is assigned to", constraint_feature,"\n")


# For each feature, we create a randomized array, except for the constrained one where the value is unique
population = pd.DataFrame(np.zeros((population_size,n_features)),
                          columns=features_names)

for column_name in features_names:
  if column_name!= constraint_feature:
    population[column_name] = np.random.uniform(df_describe.loc["min",column_name], df_describe.loc["max",column_name], population_size)
  else:
    population[column_name] = np.ones(population_size)*constraint_feature_value

display(population)

In [ ]:
target = 42

population["Y"] = RFR.predict(population)
population["target_distance"]=abs(population["Y"]-np.ones(population_size)*target)

population

In [ ]:
population.sort_values("target_distance").head(10)